In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `cleaned_merged_cc_election.csv` data from the `Resources/Cleaned_CC_CSV/Final_CSV` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame

# Define the path to your CSV file
csv_file_path = Path('/Users/yegu/Documents/Project_4_Machine_learning_YG/Cleaned_CC_CSV/Sept27_CSV/Combined_Cleaned_CSV_cc_election/cleaned_merged_cc_election_with_2020_electionresult.csv')

# Read the CSV file 
df_cc_election_votes = pd.read_csv(csv_file_path)

# Now, the empty cells will not be represented as NaN in the DataFrame

# Review the DataFrame
df_cc_election_votes.sort_values(by='STATE', ascending=True)

,STATE,YEAR,STNAME,TOT_MALE 15-49,TOT_MALE 50+,TOT_FEMALE 15-49,TOT_FEMALE 50+,WA_MALE 15-49,WA_MALE 50+,WA_FEMALE 15-49,...,NH_FEMALE 15-49,NH_FEMALE 50+,H_MALE 15-49,H_MALE 50+,H_FEMALE 15-49,H_FEMALE 50+,votepercentDEMOCRAT,votepercentREPUBLICAN,party_won,Election_Winner
305,1,2000,alabama,948961,723520,970162,879279,678301,574154,660836,...,953358,873459,26537,5948,16804,5820,0.415893,0.564652,REPUBLICAN,0
65,1,2016,alabama,934280,961302,952796,1112359,628191,732190,616035,...,909701,1094521,52589,20523,43095,17838,0.343580,0.620830,REPUBLICAN,0
151,1,2012,alabama,942627,922382,959295,1067411,645318,708174,628679,...,917611,1053540,55385,16187,41684,13871,0.384329,0.606624,REPUBLICAN,0
164,1,2008,alabama,936594,847964,941712,1009669,651288,661054,629460,...,915923,998708,36345,11986,25789,10961,0.387404,0.603169,REPUBLICAN,0
204,1,2004,alabama,932204,779822,941659,936372,658174,612954,636800,...,920889,928553,31019,8382,20770,7819,0.368444,0.624607,REPUBLICAN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,56,2000,wyoming,109224,86286,104514,90787,103202,83748,98863,...,97264,87431,8052,3437,7250,3356,0.276990,0.677565,REPUBLICAN,0
90,56,2016,wyoming,120587,118333,109988,119797,110478,112840,101096,...,97471,112852,14334,7391,12517,6945,0.218774,0.681726,REPUBLICAN,0
108,56,2012,wyoming,120009,115418,108933,117463,110256,110380,100400,...,97324,111055,13669,6667,11609,6408,0.278189,0.686426,REPUBLICAN,0
220,56,2004,wyoming,106353,96093,100748,99889,99805,93035,94709,...,93084,95976,8808,4060,7664,3913,0.290747,0.688618,REPUBLICAN,0


In [3]:
#drop the STNAME and party_won column
df_lr=df_cc_election_votes.drop(columns=['STNAME','party_won'])
df_lr

,STATE,YEAR,TOT_MALE 15-49,TOT_MALE 50+,TOT_FEMALE 15-49,TOT_FEMALE 50+,WA_MALE 15-49,WA_MALE 50+,WA_FEMALE 15-49,WA_FEMALE 50+,...,NH_MALE 50+,NH_FEMALE 15-49,NH_FEMALE 50+,H_MALE 15-49,H_MALE 50+,H_FEMALE 15-49,H_FEMALE 50+,votepercentDEMOCRAT,votepercentREPUBLICAN,Election_Winner
0,56,2020,118560,121387,107599,120335,108134,115622,98418,114586,...,113524,94888,112855,14799,7863,12711,7480,0.263879,0.694998,0
1,47,2020,1367375,1393501,1353192,1550693,1046105,1170795,1017474,1274952,...,1349658,1263543,1511493,102274,43843,89649,39200,0.374514,0.606603,0
2,36,2020,4033040,4042015,3995264,4584642,2718640,3003871,2615963,3299508,...,3444493,3134479,3892210,906451,597522,860785,692432,0.603919,0.374613,1
3,37,2020,2070745,2095415,2042064,2362670,1418891,1613063,1359952,1761349,...,1977820,1807250,2253915,256499,117595,234814,108755,0.121466,0.124836,0
4,16,2020,378695,361364,364030,372891,347908,344297,334137,354741,...,333793,308479,346988,60550,27571,55551,25903,0.330913,0.638856,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,29,2000,1204434,917911,1206511,1096942,1027607,829408,1013743,979160,...,908075,1178254,1086597,34414,9836,28257,10345,0.470843,0.504228,0
302,50,2000,129539,107563,130514,121934,125208,105698,126215,119934,...,106999,129121,121294,1409,564,1393,640,0.506347,0.406972,1
303,12,2000,3298352,2974444,3232749,3500864,2614251,2641710,2505295,3088366,...,2613982,2600694,3063195,693165,360462,632055,437669,0.488439,0.488405,1
304,16,2000,290323,205306,278632,221525,276712,199851,265538,215354,...,198208,255309,215289,29288,7098,23323,6236,0.276378,0.671696,0


### Step 2: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [4]:
# Check the balance of our target values
df_lr["Election_Winner"].value_counts()

0    161
1    145
Name: Election_Winner, dtype: int64

### Step 3: Split the data into training and testing datasets by using `train_test_split`. Train on 2000-2016 and Test on 2020 

In [5]:
# Filter data for the training set (2000-2016) and the testing set (2020)
training_data = df_lr[df_lr['YEAR'].between(2000, 2016)]
testing_data = df_lr[df_lr['YEAR'] == 2020]

# Separate the dependent variable (Y) and independent variables (X) for both sets
X_train = training_data.drop(columns=['Election_Winner', 'YEAR'])  
y_train = training_data['Election_Winner']

X_test = testing_data.drop(columns=['Election_Winner', "YEAR"])    
y_test = testing_data['Election_Winner']

# Split the data into training and testing sets (you can adjust the test_size as needed)
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Now you have X_train, y_train for training (2000-2016), and X_test, y_test for testing (2020)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [20]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the Logistic Regression model
lr_model = LogisticRegression(solver='liblinear', random_state=42)

# Fit the model using training data
lr_model.fit(X_train_scaled, y_train)


LogisticRegression(random_state=42, solver='liblinear')

###  Step 2: Save the predictions on the testing data labels by using the testing feature data (X_test) and the fitted model.

In [21]:
# Make predictions on the test set
testing_predictions = lr_model.predict(X_test_scaled)
testing_predictions_df = pd.DataFrame({"Prediction": testing_predictions, "Actual": y_test})
#pd.DataFrame({"Prediction": testing_predictions, "Actual": y_test})
testing_predictions_df

,Prediction,Actual
0,0,0
1,0,0
2,1,1
3,0,0
4,0,0
5,0,0
6,1,1
7,0,0
8,0,0
9,1,1


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [22]:
# Print the balanced_accuracy score of the model
print( f'The balanced accuracy score of the model is : {balanced_accuracy_score(y_test, testing_predictions)}')

The balanced accuracy score of the model is : 0.9407692307692308


In [23]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, testing_predictions)

array([[23,  2],
       [ 1, 25]])

In [24]:
# Print the classification report for the model

print(classification_report(y_test, testing_predictions))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94        25
           1       0.93      0.96      0.94        26

    accuracy                           0.94        51
   macro avg       0.94      0.94      0.94        51
weighted avg       0.94      0.94      0.94        51



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (Republican) and `1` (Democrat) winner in each state? winner_mapping = {'DEMOCRAT': 1, 'REPUBLICAN': 0}

**Answer:** The logistic regression model is be doing very well in predicting the both the `0` (Republican) and `1` (Democrat) winner in each state with precision, recall, and f1-score of 0.88. 

In [25]:
# Define a mapping dictionary 
#winner_mapping = {'DEMOCRAT': 1, 'REPUBLICAN': 0}


# Determine Which States are Swing States

In [26]:
# Predict probabilities for the test set
y_probabilities = pd.DataFrame(lr_model.predict_proba(X_test_scaled))

In [27]:
y_probabilities_df=y_probabilities.rename(columns={0:"Republic_Prob",1:"Democrat_Prob"})

In [28]:
y_probabilities_df

,Republic_Prob,Democrat_Prob
0,9.999936e-01,0.000006
1,9.987896e-01,0.001210
2,1.804281e-03,0.998196
3,6.143909e-01,0.385609
4,9.998121e-01,0.000188
5,9.999122e-01,0.000088
6,1.893660e-05,0.999981
7,9.018896e-01,0.098110
8,9.838379e-01,0.016162
9,6.533248e-03,0.993467


In [29]:
# Reset the indices of the test set
y_probabilities_df.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

#y_probabilities_df['Index'] = X_test.index  # Add the index column from X_test

# Assuming you have a reference to the original test data in 'original_test_data'
# and it includes a 'STATE' column
df_probability_vote_with_state = pd.concat([y_probabilities_df, X_test['STATE']], axis=1)

# Reset the DataFrame index
df_probability_vote_with_state.reset_index(drop=True, inplace=True)


# Print the DataFrame with STATE column
print("Predicted Probabilities for each class (in percentages) with STATE column:")
df_probability_vote_with_state


Predicted Probabilities for each class (in percentages) with STATE column:


,Republic_Prob,Democrat_Prob,STATE
0,9.999936e-01,0.000006,56
1,9.987896e-01,0.001210,47
2,1.804281e-03,0.998196,36
3,6.143909e-01,0.385609,37
4,9.998121e-01,0.000188,16
5,9.999122e-01,0.000088,38
6,1.893660e-05,0.999981,15
7,9.018896e-01,0.098110,39
8,9.838379e-01,0.016162,40
9,6.533248e-03,0.993467,41


In [30]:
df_2020_State= df_cc_election_votes[df_cc_election_votes['YEAR'] == 2020][['STATE', 'STNAME']]
df_2020_State

,STATE,STNAME
0,56,wyoming
1,47,tennessee
2,36,new york
3,37,north carolina
4,16,idaho
5,38,north dakota
6,15,hawaii
7,39,ohio
8,40,oklahoma
9,41,oregon


In [31]:
merged_df_predit_election_vote_state_prob = pd.merge(df_probability_vote_with_state, df_2020_State, on='STATE', how='inner')
merged_df_predit_election_vote_state_prob

,Republic_Prob,Democrat_Prob,STATE,STNAME
0,9.999936e-01,0.000006,56,wyoming
1,9.987896e-01,0.001210,47,tennessee
2,1.804281e-03,0.998196,36,new york
3,6.143909e-01,0.385609,37,north carolina
4,9.998121e-01,0.000188,16,idaho
5,9.999122e-01,0.000088,38,north dakota
6,1.893660e-05,0.999981,15,hawaii
7,9.018896e-01,0.098110,39,ohio
8,9.838379e-01,0.016162,40,oklahoma
9,6.533248e-03,0.993467,41,oregon


In [34]:
merged_df_predit_election_vote_state_prob.sort_values(by='STNAME', ascending=True, inplace=True)

In [35]:
merged_df_predit_election_vote_state_prob.head()

,Republic_Prob,Democrat_Prob,STATE,STNAME
39,9.996618e-01,0.000338,1,alabama
42,9.587736e-01,0.041226,2,alaska
44,4.931742e-01,0.506826,4,arizona
45,7.275154e-01,0.272485,5,arkansas
38,5.603967e-07,0.999999,6,california


In [36]:
#save the dataframe to excel

# Specify the folder path and Excel file name
folder_path = '/Users/yegu/Documents/Project_4_Machine_learning_YG/Sept27/Prediction_Result/Election_Result_Prediction.xlsx'


# Use the to_excel method to save the DataFrame to the Excel file
merged_df_predit_election_vote_state_prob.to_excel(folder_path, index=False)  

print(f"election prediction probabilty saved to {folder_path}")

election prediction probabilty saved to /Users/yegu/Documents/Project_4_Machine_learning_YG/Sept27/Prediction_Result/Election_Result_Prediction.xlsx


---